In [ ]:
!pip install python-binance

In [18]:
import sqlalchemy
import pandas as pd
from binance.client import Client
import asyncio
import datetime
import nest_asyncio
import pandas as pd
from binance import AsyncClient, BinanceSocketManager

api_key = "ncaGKCQNu9KS8kEwF3Jri37YjsOkKCfLGiKZcq87iThEU3QZIWdxOyeQLUvOCPpG"
api_secret = "tvSQ36Idx7W1QwiMx5ycduhziLQZaHUbhnHZGhqlSRaF6VrkP8QqsLY3LCidthdJ"

client = Client(api_key, api_secret)

In [9]:
pair = "ADAUSDT"

In [10]:
engine = sqlalchemy.create_engine("sqlite:////content/drive/MyDrive/Colab Notebooks/DB/"+pair+"stream.db")

In [16]:
nest_asyncio.apply()

In [26]:
async def main():
    client = await AsyncClient.create()
    bm = BinanceSocketManager(client)
    # start any sockets here, i.e a trade socket
    ts = bm.trade_socket('ADAUSDT')
    # then start receiving messages
    async with ts as tscm:
        while True:
            res = await tscm.recv()
            frame = createframe(res)
            frame.to_sql('ADAUSDT', engine, if_exists='append', index=False)
            print(frame)

    await client.close_connection()

if __name__ == "__main__":
    loop = asyncio.get_event_loop()
    loop.run_until_complete(main())

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
    symbol                    Time   Price
0  ADAUSDT 2022-06-05 22:12:07.978  0.5611
    symbol                    Time   Price
0  ADAUSDT 2022-06-05 22:12:07.978  0.5611
    symbol                    Time   Price
0  ADAUSDT 2022-06-05 22:12:07.978  0.5611
    symbol                    Time   Price
0  ADAUSDT 2022-06-05 22:12:07.978  0.5611
    symbol                    Time   Price
0  ADAUSDT 2022-06-05 22:12:07.978  0.5611
    symbol                    Time   Price
0  ADAUSDT 2022-06-05 22:12:07.978  0.5611
    symbol                    Time   Price
0  ADAUSDT 2022-06-05 22:12:07.978  0.5611
    symbol                    Time   Price
0  ADAUSDT 2022-06-05 22:12:07.978  0.5611
    symbol                    Time   Price
0  ADAUSDT 2022-06-05 22:12:13.048  0.5612
    symbol                    Time   Price
0  ADAUSDT 2022-06-05 22:12:13.048  0.5612
    symbol                    Time   Price
0  ADAUSDT 2022-06-05 22:12:13.048  0.5612
    symbol        

CANCEL read_loop


    symbol                    Time  Price
0  ADAUSDT 2022-06-05 22:33:04.885  0.559
    symbol                    Time  Price
0  ADAUSDT 2022-06-05 22:33:04.885  0.559


KeyboardInterrupt: ignored

In [14]:
def createframe(msg):
  df = pd.DataFrame([msg])
  df = df.loc[:, ['s', 'E', 'p']]
  df.columns = ['symbol', 'Time', 'Price']
  df.Price = df.Price.astype(float)
  df.Time = pd.to_datetime(df.Time, unit='ms')
  time_change = datetime.timedelta(hours=9)
  df.Time = df.Time + time_change 
  return df

In [27]:
df = pd.read_sql(pair, engine)
df

,symbol,Time,Price
0,ADAUSDT,2022-06-05 22:02:36.607,0.5604
1,ADAUSDT,2022-06-05 22:02:36.962,0.5604
2,ADAUSDT,2022-06-05 22:02:37.322,0.5604
3,ADAUSDT,2022-06-05 22:02:38.530,0.5604
4,ADAUSDT,2022-06-05 22:02:38.887,0.5604
...,...,...,...
3791,ADAUSDT,2022-06-05 22:33:07.776,0.5587
3792,ADAUSDT,2022-06-05 22:33:07.832,0.5587
3793,ADAUSDT,2022-06-05 22:33:10.043,0.5588
3794,ADAUSDT,2022-06-05 22:33:10.044,0.5587


In [ ]:
df['Benchmark'] = df.Price.cummax()
df['TSL'] = df.Benchmark * 0.95
df

In [ ]:
def strategy(entry, lookback, qty, open_position=False):
  while True:
    df = pd.read_sql(pair, engine)
    lookbackperiod = df.iloc[-lookback:]
    cumret = (lookbackperiod.Price.pct_change() + 1).cumprod() - 1
    if cumret[cumret.last_valid_index()] < entry:
      order = client.create_order(symbol=pair, side='BUY', type='MARKET', quantity=qty)
      print(order)
      open_position=True
      break
    #Trailing Stop Loss part from here on
  if open_position:
    while True:
      df = pd.read_sql(f"""SELECT * FROM {pair} WHERE \
      Time >= '{pd.to_datetime(order['transactTime'], unit='ms')}'""", engine)
      df['Benchmark'] = df.Price.cummax()
      df['TSL'] = df['Benchmark'] * 0.95 
      #0.995(In case it doesn't execute very quickly)
      if df[df.Price < df.TSL].first_valid_index():
        order = client.create_order(symbol=pair, side='SELL', type='MARKET', quantity=qty)
        print(order)
        break

In [ ]:
strategy(-0.0015, 60, 50)